In [1]:
print("Started")
#!/usr/bin/env python
# coding: utf-8

# # WWTP Prediction
# ###### Made by Nitzan Farhi
import os
import random
import sys
import pandas as pd
import seaborn as sns
from keras.optimizers import SGD
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import metrics
import preprocess
from metrics import *
from imblearn.over_sampling import RandomOverSampler
import lstme
import importlib
#import sherpa
import csv
importlib.reload(lstme)
import math
import models
importlib.reload(lstme)
importlib.reload(models)
from sklearn.model_selection import train_test_split


Started


In [2]:
#df = preprocess.preprocess_df(clabel.AMMONIA, graphs=True)
#df.to_pickle("temp_dataframe.pkl") 

#import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
#df.describe()

sns.set(style='white', context='notebook', palette='deep')
# Seed value
seed_value = 0
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

Ytest, Yhat = [], []

# In[439]:

In [3]:
def normalize(X, Y,binary=False):
    scalers = list()
    for i in tqdm(range(X.shape[1])):
        scalers.append(MinMaxScaler())
        X[:, i, :] = scalers[i].fit_transform(X[:, i, :])

    scalers.append(MinMaxScaler())
    if not binary:
        Y = scalers[i + 1].fit_transform(Y)
    return X, Y, scalers


def normalize_2(x,y, binary=False):
    x_min = x.min(axis=(0, 1), keepdims=True)
    x_max = x.max(axis=(0, 1), keepdims=True)
    x = (x - x_min)/(x_max - x_min)
    y_min = y.min(axis=(0, 1), keepdims=True)
    y_max = y.max(axis=(0, 1), keepdims=True)
    y = (y - y_min)/(y_max -y_min)
    
    return np.nan_to_num(x),np.nan_to_num(y),(x_min,x_max,y_min,y_max)

# Assert values are fine
def check_boundaries(vals, min_val, max_val):
    # print(np.amin(vals),np.amax(vals))
    if np.amin(vals) >= min_val and np.amax(vals) <= max_val:
        return True
    return False

def oversampler(conX,conY):
    shape2 = conX.shape[2]
    conX = conX.reshape(conX.shape[0],-1)  # CNN_LSTM
    oversample = RandomOverSampler(sampling_strategy='minority')
    # fit and apply the transform
    conX, conY = oversample.fit_resample(conX, conY)
    conX = conX.reshape(conX.shape[0],-1,shape2)
    return conX,conY

def series_to_supervised(sequences, n_steps_in=1, n_steps_out=1, steps = 1, binary=False):
    X, Y = list(), list()
    print("LIMIT IS: ",ammonia_limit,"Steps are: ",steps," Size is ",len(sequences))
    
    for i in range(0, len(sequences),steps):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[out_end_ix - 1:out_end_ix][:, 0]
        
        if binary:
            seq_y = [float(seq_y > ammonia_limit)]
        if True or sufficient_feed_flow(seq_x):
            X.append(seq_x)
            Y.append(seq_y)
    return np.array(X), np.array(Y)

In [4]:
def add_one_hot(ret_df):    
    day_names = pd.get_dummies(ret_df.index.weekday,prefix='weekday')
    hours = pd.get_dummies(ret_df.index.hour, prefix='hour')
    ret_df = ret_df.reset_index()
    tmp = pd.concat((ret_df, hours, day_names), axis=1)
    
    ret_df = ret_df.reset_index()
    ret_df = pd.concat([ret_df, day_names, hours], axis=1)
    return ret_df


def split_by_seasons(mdf):
    begin = 0
    for i in range(mdf.shape[0] - 1):
        if (mdf.index[i + 1] - mdf.index[i]).days > 50:
            ret_df = mdf.iloc[begin:i, :]
            print("Time Period: ",mdf.index[begin+1],"->",mdf.index[i])
            begin = i
            yield ret_df
    print("Time Period: ",mdf.index[begin+1],"->",mdf.index[i])
    yield mdf.iloc[begin:i, :]
    
    
def find_max_f1():
    mmax = 0
    ival = 0
    for i in range(1, 200):
        check_y_hat =  Yhat*(1+0.01*i)
        rmse = calc_rmse(Ytest,check_y_hat, graph=False)
        tn, fp, fn, tp  = metrics.calc_tp_fp_rate(Ytest,check_y_hat,selected_value,binary=False,graph=False)
        precision = tp / (tp + fp) 
        recall =  tp / (tp + fn)
        f1 = 2 * (precision * recall) / (precision + recall)
        if f1 > mmax:
            mmax = f1
            ival = i
    return ival,mmax

def aggregate_data(df,aggr,back,out):
    HOUR = 60
    
    df = df.resample("%dMin" % aggr).mean()
    aggr = HOUR / aggr # aggr = 2
    back = int(back * aggr)
    out = int(out* aggr)
        
    return df,back,out


def geometric_series(N, un0, q):
    u = np.empty((N,))
    u[0] = un0
    u[1:] = q
    return np.round(np.cumprod(u),0).astype(int)

def geometric_series_with_back(back,un0,q):
    for i in range(1,back):
        if sum(geometric_series(i,un0,q))>back:
            return geometric_series(i-1,un0,q)
    return geometric_series(back-1,un0,q)



In [5]:
X1,Y1=[],[]
def create_final_dataset(df,perc_df=False,binary=False,**kwargs):
    global first_reactor_index,mdfs,X1,Y1
    mdfs = []
    X,Y = [],[]
    for mdf in [df]:

                print("[LOG] Data Aggregation")
                #mdf, uback,uout = aggregate_data(mdf,kwargs["first_aggr"],kwargs["back"],kwargs["out"])
                uback,uout=    kwargs["back"],kwargs["out"]
                if type(perc_df) != type(False):
                    print("[LOG] Additional Dataset Merge")

                    df_min,df_max = min(mdf.index),max(mdf.index)

                    cur = perc_df[df_min:df_max]
                
                    mdf["date_hour"]=mdf.index
                    mdf = mdf.merge(cur,on="date_hour",how='left')
                    mdf.index = mdf["date_hour"]
                    mdf = mdf.drop(columns=['date_hour'])

                    mdf = add_one_hot(mdf)
                    mdf = mdf.iloc[:,2:]

                    mdf = mdf.fillna(-1)
                    mdf = mdf.replace('-',-1)

                    first_reactor_index = mdf.columns.get_loc('FeedFlowtoReactor1m3/hr')
                mdfs.append(mdf)
                #print("[LOG] series to supervised back ",uback," out ", uout)
                #X1,Y1 = series_to_supervised(mdf.values, n_steps_in=uback , n_steps_out=uout,
                #                            steps=kwargs["steps"],binary=binary) 
                #comment = ""
                #print("[LOG] Data Linearization")
                #X1,Y1, comment = lstme.linearize_dataset(mdf.values,steps_out=uout,size=uback,
                #                           binary=binary,**kwargs)
                X1,Y1, comment = lstme.special_linearize(mdf.values,steps_out=uout,size=uback,
                                           binary=binary,**kwargs)
                
                X.append(X1)
                Y.append(Y1)
                print("[LOG] Finished season")

    return X,Y, comment

def param_dict_generator():
    for model in list(models.ModelType):
        for optimizer in ['SGD',  'Adam']:
            for epoch in  [50, 100]:
                for batch in [20]:
                    yield {'optimizer':optimizer,'model':model,'epoch':epoch,'batch':batch}

In [6]:
def calc_series_size_2(mul,init,back):
    if mul == 1:
        return int(back/init)
    result = back * (mul-1)
    result = result / init
    result+=1
    result = math.log(result,mul)
    return int(result)

def calc_needed_back(mul,init,entrances):
    back = 1
    calced_size = calc_series_size(mul,init,back)
    while calced_size < entrances:
        back+=10
        calced_size = calc_series_size(mul,init,back)

    return back-1


In [7]:
X,Y,model,val,Yval,Yhat,Ytest = [],[],[],[],[],[],[]
def run_exp(exp_params,perc_df=False):
    global X,Y,df,model,Xval,Yval,Yhat,Ytest
    print("[LOG] Creating Dataset")
    X,Y,comments = create_final_dataset(df[:],perc_df=perc_df, binary=binary,**exp_params)
    exp_params["comments"]+=str(comments)
    print("[LOG] Normalizing")
    conX,conY = np.concatenate(X),np.concatenate(Y)
    conX = np.asarray(conX).astype(np.float32)
    conY = np.asarray(conY).astype(np.float32)
    conX, conY, scalers = normalize(conX,conY ,binary=binary)  

    print("[LOG] Spliting Data")

    Xtrain, Xval, Ytrain, Yval = train_test_split(conX, conY, test_size=0.2, random_state=42)
    #if binary:
    #    Xtrain,Ytrain = oversampler(Xtrain,Ytrain)
    # 0.25 x 0.8 = 0.2
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xtrain, Ytrain, test_size=0.25, random_state=42)

    model = models.make_model(exp_params['model'], conX, conY,opt=exp_params['optimizer'], binary=binary)
    print("[LOG] Training")
    if np.isnan(np.min(conX)):
        print("ERROR! contains nan")
        return 

    history = models.fit(Xtrain, Ytrain, Xtest, Ytest,model,
                         epochs=exp_params['epoch'],batch_size= exp_params['batch'],verbose=1 if exp_params["graph"] else 0,graph=exp_params["graph"])
    # In[7]
    Yhat, Ytest = models.evaluate(model, Xval, Yval, scalers, binary=binary)
    if not binary:
        rmse = metrics.calc_rmse(Ytest,Yhat,graph=exp_params["graph"])
        exp_params["rmse"] = rmse
    else:
        accuracy = metrics.calc_best_accuracy(Ytest,Yhat)
        f1 = metrics.calc_best_f1(Ytest,Yhat)
        _,_,_,auc = metrics.roc(Ytest,Yhat,graph=exp_params["graph"])
        print(accuracy,f1,auc)
        exp_params["accuracy"] = accuracy
        exp_params["f1"] = f1
        exp_params["auc"] = auc

        
    return exp_params


def run_exp_variable(key_name,values):
    res = []
    print("START")
    importlib.reload(models)


    #Multiplier = [round(1+i*0.1,1) for i in range(10)]
    m_models = list(models.ModelType)
    #key_name = 'model'
    #values = m_models
    result_list = []
    with open("LSTME/%s/%s_%s" % (experiment_name,var_name,key_name),"w+", newline='') as mfile:
        w = csv.DictWriter(mfile, template_params.keys())
        w.writeheader()
        for exp_params in [dict(template_params) for i in values if not template_params.update({key_name:i})]:
                print(exp_params)
                return_list.append(run_exp(exp_param))
    return result_list

   
def gen_num_to_locations(gen_num,back):
        gen_num = [int(i) for i in gen_num.tolist()]
        gen_num.append(back-sum(gen_num))
        res_arr = []
        cur = 0
        for var in gen_num:
            res_arr.append(cur+var)
            cur=cur+var
        return res_arr
    
def run_func(gen_num):
        global template_params
        print(gen_num)
        res_arr = gen_num_to_locations(gen_num,template_params["back"])
        template_params["gen_num"] = res_arr
        res = run_exp(dict(template_params),perc_df=perc_df)

        return res

def series_maker(upper_limit):
    return "".join(list(map(str,range(1,upper_limit))))


In [ ]:

importlib.reload(lstme)
importlib.reload(models)
import pathlib


keys = ["first_aggr", "steps", "back", "out", "model", "optimizer", "limit",
        "multiplier", "init_size", "method", "epoch", "batch", "rmse",
        "accuracy", "f1", "auc", "comments", "graph", "gen_num", "kfold"]

kfold_num = 5
ammonia_limit = 15
epochs = 100
comment = ""
perc_df = False
datasets = []
datasets = ["ammonia"]

for dataset_name in datasets:
    wanted_size = 5000
    out_predict = 4

    print("Experiment Started - ", dataset_name)
    variant = "complicated"
    mode = "a"
    pathlib.Path("LSTME/" + variant).mkdir(parents=True, exist_ok=True)

    if dataset_name == "turkey":
        adf = pd.read_pickle("datasets/turkey_power.pkl")
        mfile = open("LSTME/%s/turkey_power.csv" % variant, mode, newline='')

    elif dataset_name == "nitrate":
        adf = pd.read_pickle("nitrate_new.pkl")
        mfile = open("LSTME/%s/nitrate.csv" % variant, mode, newline='')
        
    elif dataset_name == "ammonia":
        adf = pd.read_pickle("temp_dataframe.pkl")
        mfile = open("LSTME/%s/ammonia.csv" % variant, mode, newline='')

    else:
        raise Exception()
        
        
    w = csv.DictWriter(mfile, keys)
    if (mode == "w"):
        w.writeheader()

    binary = False
    wanted_size = adf.shape[0] // kfold_num

    rmse_list = []
    series = []

    window_size = 8

    for window_size in range(5,10,2):
        # This is where parameters are selected
        # To do multiple experiments, change template_params every time
        template_params = {"first_aggr": 1, "steps": 1, "back": window_size, "out": out_predict,
                           "model": models.ModelType.STACKED_LSTM, "optimizer": "SGD",
                           "limit": ammonia_limit, "multiplier": 1, "init_size": 1, "method": aggr_type,
                           "epoch": epochs, "batch": 32, "rmse": "", "accuracy": "", "f1": "", "auc": "",
                           "comments": f"{idx}: {comment} ",
                           "graph": False, "gen_num": "", "kfold": kfold_num}

        # ignore this param
        series = np.array([1]*window_size)
        res = run_func(series)
        
        print(res)
        if res is None:
            print("Returned Error")
            continue
        final_dict = res
        try:
            rmse_list.append(final_dict["rmse"])
        except:
            traceback.print_exc()
            pass

        if w is not None:
            w.writerow(final_dict)
            mfile.flush()

mfile.close()


Experiment Started -  turkey
[[1, 2, 3, 4, 5, 6, 7], [1, 1, 1, 2, 2, 2, 3], [1, 1, 1, 2, 4, 8, 15], [1, 1, 1, 2, 2, 2, 10], [4, 4, 4, 4, 4, 4, 4], [2, 2, 2, 2, 3, 3, 3], [1, 1, 1, 1, 1, 2, 2], [1, 1, 1, 2, 2, 2, 2]]
[1 1 1 2 2 2 2]
[LOG] Creating Dataset
[LOG] Data Aggregation
[LOG]Started Linearizing]


100%|██████████| 8/8 [00:00<00:00, 166.56it/s]


[LOG] Finished season
[LOG] Normalizing
[LOG] Spliting Data
ModelType.STACKED_LSTM


MODEL: STACKED_LSTM  LossFunc:  mse  OptimizerL  SGD
[LOG] Training
{'first_aggr': 1, 'steps': 1, 'back': 48, 'out': 4, 'model': <ModelType.STACKED_LSTM: 2>, 'optimizer': 'SGD', 'limit': 15, 'multiplier': 1, 'init_size': 1, 'method': 'avg', 'epoch': 100, 'batch': 32, 'rmse': 3110.044372673805, 'accuracy': '', 'f1': '', 'auc': '', 'comments': '0:  Window Sizes: [8], [1, 1, 1, 2, 2, 2, 2, 37]', 'graph': False, 'gen_num': [1, 2, 3, 5, 7, 9, 11, 48], 'kfold': 5}
[1 1 1 1 1 2 2]
[LOG] Creating Dataset
[LOG] Data Aggregation
[LOG]Started Linearizing]


100%|██████████| 8/8 [00:00<00:00, 165.23it/s]

...


[LOG] Finished season
[LOG] Normalizing
[LOG] Spliting Data
ModelType.STACKED_LSTM


MODEL: STACKED_LSTM  LossFunc:  mse  OptimizerL  SGD
[LOG] Training
{'first_aggr': 1, 'steps': 1, 'back': 48, 'out': 4, 'model': <ModelType.STACKED_LSTM: 2>, 'optimizer': 'SGD', 'limit': 15, 'multiplier': 1, 'init_size': 1, 'method': 'avg', 'epoch': 100, 'batch': 32, 'rmse': 3108.3732079658644, 'accuracy': '', 'f1': '', 'auc': '', 'comments': '5:  Window Sizes: [8], [1, 1, 1, 2, 4, 8, 15, 16]', 'graph': False, 'gen_num': [1, 2, 3, 5, 9, 17, 32, 48], 'kfold': 5}
[1 1 1 2 2 2 3]
[LOG] Creating Dataset
[LOG] Data Aggregation
[LOG]Started Linearizing]


100%|██████████| 8/8 [00:00<00:00, 181.51it/s]


[LOG] Finished season
[LOG] Normalizing
[LOG] Spliting Data
ModelType.STACKED_LSTM


MODEL: STACKED_LSTM  LossFunc:  mse  OptimizerL  SGD
[LOG] Training
